# Reliability anaysis


## Packages and Helpers

In [1]:
# packages
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from utils import load_dataframes
from scipy.stats import pearsonr
import random
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import pingouin as pg
import seaborn as sns

## Read Processed Data

In [2]:
# load overall df
random_all_data = pd.read_csv("processed_data/items_per_LLM_random_simulation.csv")
semi_random_all_data = pd.read_csv("processed_data/items_per_LLM_semi_random_simulation.csv")


## Normalize item scores where tasks have different scales on different items
- AUDIT, FTND, GABS, PG

In [3]:
def normalize_per_scale(data):
    scales_to_normalize = ["AUDIT scale", "FTND scale", "GABS scale", "PG scale"]

    data = data.copy()

    # Loop over each experiment you want to normalize
    for scale in scales_to_normalize:
        mask = data["experiment"] == scale

        if mask.sum() == 0:
            continue  # skip if scale not present

        # Normalize score
        data.loc[mask, "score"] = (
            data.loc[mask, "score"] - data.loc[mask, "score"].min()
        ) / (data.loc[mask, "score"].max() - data.loc[mask, "score"].min())

        # Normalize score_top_n
        data.loc[mask, "score_top_n"] = (
            data.loc[mask, "score_top_n"] - data.loc[mask, "score_top_n"].min()
        ) / (data.loc[mask, "score_top_n"].max() - data.loc[mask, "score_top_n"].min())

    return data

random_all_data = normalize_per_scale(random_all_data)
semi_random_all_data = normalize_per_scale(semi_random_all_data)


## Chronbach's Alpha

In [4]:
# function:
def compute_cronbach_alpha(data, score = "score"):
    results = []

    # loop through experiments
    for exp, exp_data in data.groupby("experiment"):
        
        # check whether this experiment has subcategories
        if exp_data["category"].notna().any() and exp != "SOEP scale":
            # compute alpha per category (domain)
            for domain, domain_data in exp_data.groupby("category"):
                df_wide = domain_data.pivot_table(
                    index="model", columns="item", values=score
                )
                #print(df_wide)
                if df_wide.shape[1] > 1:
                    alpha, ci = pg.cronbach_alpha(df_wide)
                else:
                    alpha, ci = None, (None, None)
                
                results.append({
                    "experiment": exp,
                    "domain": domain,
                    "alpha": alpha,
                    "alpha_CI": ci
                })
        else:
            # compute alpha for the whole experiment
            df_wide = exp_data.pivot_table(
                index="model", columns="item", values=score
            )
            #print(exp, df_wide.std().describe())
            if df_wide.shape[1] > 1:
                alpha, ci = pg.cronbach_alpha(df_wide)
            else:
                alpha, ci = None, (None, None)
            
            results.append({
                "experiment": exp,
                "domain": "total",  # no subdomain
                "alpha": alpha,
                "alpha_CI": ci
            })

    # convert to DataFrame
    return(pd.DataFrame(results))



In [5]:
# ---- Cronbach's alpha per domain or directly per scale, depending on whether the scale has subdomains ----
alpha_df_random = compute_cronbach_alpha(random_all_data)
alpha_df_semi_random = compute_cronbach_alpha(semi_random_all_data)


# ---- Cronbach's alpha per domain or directly per scale, depending on whether the scale has subdomains ----
alpha_df_random_top_n= compute_cronbach_alpha(random_all_data, score = "score_top_n")
alpha_df_semi_random_top_n = compute_cronbach_alpha(semi_random_all_data, score = "score_top_n")



# Merge only selected columns
alpha_df = alpha_df_random.merge(
    alpha_df_semi_random,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)

display(alpha_df)

,experiment,domain,alpha_random,alpha_CI_random,alpha_semi_random,alpha_CI_semi_random
0,AUDIT scale,total,0.185702,"[-0.213, 0.495]",0.125090,"[-0.303, 0.457]"
1,BARRAT scale,BISa,0.085305,"[-0.377, 0.436]",-0.043888,"[-0.571, 0.357]"
2,BARRAT scale,BISm,0.188272,"[-0.209, 0.496]",-0.241641,"[-0.849, 0.23]"
3,BARRAT scale,BISn,0.014077,"[-0.469, 0.388]",0.200345,"[-0.191, 0.504]"
4,BART task,total,0.127238,"[-0.29, 0.459]",0.127238,"[-0.29, 0.459]"
5,CARE scale,CAREa,0.083517,"[-0.373, 0.434]",-0.016770,"[-0.524, 0.372]"
6,CARE scale,CAREs,0.095543,"[-0.38, 0.447]",-0.040084,"[-0.586, 0.364]"
7,CARE scale,CAREw,0.230793,"[-0.21, 0.539]",-0.237210,"[-0.945, 0.258]"
8,CCT task,total,-0.124240,"[-0.697, 0.33]",-0.124240,"[-0.697, 0.33]"
9,DAST scale,total,-0.198741,"[-0.765, 0.251]",-0.286199,"[-0.894, 0.196]"


## Split-half Reliability

In [6]:
def split_half_reliability(df_items, n_splits=100):
    """Compute average split-half reliability (Spearman-Brown corrected)."""

    k = df_items.shape[1]
    
    if k < 2:
        return None, None

    cols = df_items.columns
    results = []

    for _ in range(n_splits):
        shuffled = np.random.permutation(cols)
        half1 = shuffled[:k//2]
        half2 = shuffled[k//2:]

        s1 = df_items[half1].sum(axis=1)
        s2 = df_items[half2].sum(axis=1)

        r, _ = pearsonr(s1, s2)

        if np.isfinite(r):
            r_sb = (2 * r) / (1 + r)  # Spearman-Brown correction
            results.append(r_sb)

    if len(results) == 0:
        return None, None
    
    return np.mean(results), np.std(results)


In [7]:
def compute_split_half_reliability(data, score="score"):
    rows = []  # final list of results

    for exp, exp_data in data.groupby("experiment"):

        # case 1: experiments with domains
        if exp_data["category"].notna().any() and exp != "SOEP scale":
            for domain, domain_data in exp_data.groupby("category"):

                df_wide = domain_data.pivot_table(
                    index="model", columns="item", values=score
                )

                mean_rsb, sd_rsb = split_half_reliability(df_wide)

                rows.append({
                    "experiment": exp,
                    "domain": domain,
                    "split_half_mean": mean_rsb,
                    "split_half_sd": sd_rsb
                })

        # case 2: experiments without domains
        else:
            df_wide = exp_data.pivot_table(
                index="model", columns="item", values=score
            )

            mean_rsb, sd_rsb = split_half_reliability(df_wide)

            rows.append({
                "experiment": exp,
                "domain": "total",
                "split_half_mean": mean_rsb,
                "split_half_sd": sd_rsb
            })

    return pd.DataFrame(rows)


In [8]:
split_half_reliability_random = compute_split_half_reliability(random_all_data)
split_half_reliability_semi_random = compute_split_half_reliability(semi_random_all_data)


# Merge 
split_half_rel_df = split_half_reliability_random.merge(
    split_half_reliability_semi_random,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)

display(split_half_rel_df)

,experiment,domain,split_half_mean_random,split_half_sd_random,split_half_mean_semi_random,split_half_sd_semi_random
0,AUDIT scale,total,0.217539,0.165908,0.138413,0.172688
1,BARRAT scale,BISa,0.070332,0.168416,-0.035532,0.209921
2,BARRAT scale,BISm,0.175989,0.150816,-0.230216,0.211604
3,BARRAT scale,BISn,-0.003433,0.188046,0.198894,0.140129
4,BART task,total,0.124540,0.195315,0.088670,0.214436
5,CARE scale,CAREa,0.097298,0.138345,0.002292,0.133671
6,CARE scale,CAREs,0.037931,0.232297,-0.208403,0.350322
7,CARE scale,CAREw,0.174230,0.341446,-0.262988,0.135944
8,CCT task,total,-0.138787,0.311933,-0.153859,0.257580
9,DAST scale,total,-0.231502,0.201841,-0.329624,0.291499
